In [1]:
import numpy as np
from hyperopt import hp, fmin, tpe, space_eval
from scikits.odes.ode import ode
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from explicit_functions import dual_control, upstream_repression, no_control, downstream_activation, loss_biological

In [8]:

def activation(x, k, theta, n):
    return (k*(x/theta)**n)/(1+(x/theta)**n)

def repression(x, k, theta, n):
    return k/(1+(x/theta)**n)

def nonlinearity(x, kc, km):
    return (kc*x)/(km+x)

def dual_control(t, y, ydot, p):
    lam=1.93E-4; Vin=1.; e0=0.0467
    n1, n2 = [2., 2.]
    k1, k2, theta1, theta2 = p
    ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
    ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
    ydot[2] = repression(y[1], k1, theta1, n1) - lam*y[2]
    ydot[3] = activation(y[1], k2, theta2, n2) - lam*y[3]
    ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
    ydot[5] = repression(y[1], k1, theta1, n1) + activation(y[1], k2, theta2, n2)

def upstream_repression(t, y, ydot, p):
    lam=1.93E-4; Vin=1.; e0=0.0467
    n1, n2 = [2., 2.]
    k1, k2, theta1, theta2 = p
    ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
    ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
    ydot[2] = repression(y[1], k1, theta1, n1) - lam*y[2]
    ydot[3] = k2 - lam*y[3]
    ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
    ydot[5] = repression(y[1], k1, theta1, n1) + k2

def downstream_activation(t, y, ydot, p):
    lam=1.93E-4; Vin=1.; e0=0.0467
    n1, n2 = [2., 2.]
    k1, k2, theta1, theta2 = p
    ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
    ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
    ydot[2] = k1 - lam*y[2]
    ydot[3] = activation(y[1], k2, theta2, n2) - lam*y[3]
    ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
    ydot[5] = activation(y[1], k2, theta2, n2) + k1

def no_control(t, y, ydot, p):
    lam=1.93E-4; Vin=1.; e0=0.0467
    n1, n2 = [2., 2.]
    k1, k2, theta1, theta2 = p
    ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
    ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
    ydot[2] = k1 - lam*y[2]
    ydot[3] = k2 - lam*y[3]
    ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
    ydot[5] = k1 + k2

def loss_biological(j1, j2, alpha1=1E-5, alpha2=1E-2):
        """Computes scalarized loss including genetic constraint and product production"""
        loss = alpha1*j1 + alpha2*j2
        return j1, j2, loss


In [9]:
space = [hp.uniform('k1', 1E-7, 1E-3), hp.uniform('k2', 1E-7, 1E-3), hp.uniform('theta1', 0.001, 10), hp.uniform('theta2', 0.001, 10)]

In [10]:
#Set architecture
ode_function = dual_control
ode_name = 'dc'
max_iters = [500]
num_tries = 3


def objective(param_values):
    #Integration conditions
    t = np.linspace(0, 5E4, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.])
    extra_options = {'old_api': False, 'user_data': param_values}
    ode_solver = ode('cvode', ode_function, **extra_options)
    solution = ode_solver.solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2)
    return loss

In [11]:
perturbed_percentages = [0.5, 0.75, 0.9, 0.95, 1.05, 1.1, 1.25, 1.5]
global kc
global km
iters_list = []
km_perturbs_list = []; kc_perturbs_list = []
k1s = []; k2s = []; theta1s = []; theta2s = []
best_losses = []
for num_iters in max_iters:
    for n in range(num_tries):
        for kc_perturbation in perturbed_percentages:
            for km_perturbation in perturbed_percentages:
                kc = kc_perturbation*12.
                km = km_perturbation*10.
                best = fmin(objective, space, algo=tpe.suggest, max_evals=num_iters)
                best_losses.append(objective(list(best.values())))
                d = str(best)
                k1s.append(float(d.split(',')[0].split(':')[-1]))
                k2s.append(float(d.split(',')[1].split(':')[-1]))
                theta1s.append(float(d.split(',')[2].split(':')[-1]))
                theta2s.append(float(d.split(',')[3].split(':')[-1].split('}')[0]))
                iters_list.append(num_iters)
                kc_perturbs_list.append(kc_perturbation)
                km_perturbs_list.append(km_perturbation)

num_trials_data = pd.DataFrame({'num_iters': iters_list, 'km_perturb': km_perturbs_list, 'kc_perturb': kc_perturbs_list, 'loss': best_losses, 'k1':k1s, 'k2':k2s, 'theta1':theta1s, 'theta2':theta2s})
num_trials_data.to_csv(ode_name + '_perturbation_grid.csv')

100%|██████████| 500/500 [00:10<00:00, 49.26trial/s, best loss: 0.06113471525610552]


In [12]:
#Set architecture
ode_function = no_control
ode_name = 'nc'
max_iters = [500]
num_tries = 3


def objective(param_values):
    #Integration conditions
    t = np.linspace(0, 5E4, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.])
    extra_options = {'old_api': False, 'user_data': param_values}
    ode_solver = ode('cvode', ode_function, **extra_options)
    solution = ode_solver.solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2)
    return loss

perturbed_percentages = [0.5, 0.75, 0.9, 0.95, 1.05, 1.1, 1.25, 1.5]
global kc
global km
iters_list = []
km_perturbs_list = []; kc_perturbs_list = []
k1s = []; k2s = []; theta1s = []; theta2s = []
best_losses = []
for num_iters in max_iters:
    for n in range(num_tries):
        for kc_perturbation in perturbed_percentages:
            for km_perturbation in perturbed_percentages:
                kc = kc_perturbation*12.
                km = km_perturbation*10.
                best = fmin(objective, space, algo=tpe.suggest, max_evals=num_iters)
                best_losses.append(objective(list(best.values())))
                d = str(best)
                k1s.append(float(d.split(',')[0].split(':')[-1]))
                k2s.append(float(d.split(',')[1].split(':')[-1]))
                theta1s.append(float(d.split(',')[2].split(':')[-1]))
                theta2s.append(float(d.split(',')[3].split(':')[-1].split('}')[0]))
                iters_list.append(num_iters)
                kc_perturbs_list.append(kc_perturbation)
                km_perturbs_list.append(km_perturbation)

num_trials_data = pd.DataFrame({'num_iters': iters_list, 'km_perturb': km_perturbs_list, 'kc_perturb': kc_perturbs_list, 'loss': best_losses, 'k1':k1s, 'k2':k2s, 'theta1':theta1s, 'theta2':theta2s})
num_trials_data.to_csv(ode_name + '_perturbation_grid.csv')

100%|██████████| 500/500 [00:09<00:00, 55.45trial/s, best loss: 0.05840910492913613]


In [13]:
#Set architecture
ode_function = upstream_repression
ode_name = 'ur'
max_iters = [500]
num_tries = 3


def objective(param_values):
    #Integration conditions
    t = np.linspace(0, 5E4, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.])
    extra_options = {'old_api': False, 'user_data': param_values}
    ode_solver = ode('cvode', ode_function, **extra_options)
    solution = ode_solver.solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2)
    return loss

perturbed_percentages = [0.5, 0.75, 0.9, 0.95, 1.05, 1.1, 1.25, 1.5]
global kc
global km
iters_list = []
km_perturbs_list = []; kc_perturbs_list = []
k1s = []; k2s = []; theta1s = []; theta2s = []
best_losses = []
for num_iters in max_iters:
    for n in range(num_tries):
        for kc_perturbation in perturbed_percentages:
            for km_perturbation in perturbed_percentages:
                kc = kc_perturbation*12.
                km = km_perturbation*10.
                best = fmin(objective, space, algo=tpe.suggest, max_evals=num_iters)
                best_losses.append(objective(list(best.values())))
                d = str(best)
                k1s.append(float(d.split(',')[0].split(':')[-1]))
                k2s.append(float(d.split(',')[1].split(':')[-1]))
                theta1s.append(float(d.split(',')[2].split(':')[-1]))
                theta2s.append(float(d.split(',')[3].split(':')[-1].split('}')[0]))
                iters_list.append(num_iters)
                kc_perturbs_list.append(kc_perturbation)
                km_perturbs_list.append(km_perturbation)

num_trials_data = pd.DataFrame({'num_iters': iters_list, 'km_perturb': km_perturbs_list, 'kc_perturb': kc_perturbs_list, 'loss': best_losses, 'k1':k1s, 'k2':k2s, 'theta1':theta1s, 'theta2':theta2s})
num_trials_data.to_csv(ode_name + '_perturbation_grid.csv')

100%|██████████| 500/500 [00:08<00:00, 60.61trial/s, best loss: 0.06634595468411796]


In [14]:
#Set architecture
ode_function = downstream_activation
ode_name = 'da'
max_iters = [500]
num_tries = 3


def objective(param_values):
    #Integration conditions
    t = np.linspace(0, 5E4, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.])
    extra_options = {'old_api': False, 'user_data': param_values}
    ode_solver = ode('cvode', ode_function, **extra_options)
    solution = ode_solver.solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2)
    return loss

perturbed_percentages = [0.5, 0.75, 0.9, 0.95, 1.05, 1.1, 1.25, 1.5]
global kc
global km
iters_list = []
km_perturbs_list = []; kc_perturbs_list = []
k1s = []; k2s = []; theta1s = []; theta2s = []
best_losses = []
for num_iters in max_iters:
    for n in range(num_tries):
        for kc_perturbation in perturbed_percentages:
            for km_perturbation in perturbed_percentages:
                kc = kc_perturbation*12.
                km = km_perturbation*10.
                best = fmin(objective, space, algo=tpe.suggest, max_evals=num_iters)
                best_losses.append(objective(list(best.values())))
                d = str(best)
                k1s.append(float(d.split(',')[0].split(':')[-1]))
                k2s.append(float(d.split(',')[1].split(':')[-1]))
                theta1s.append(float(d.split(',')[2].split(':')[-1]))
                theta2s.append(float(d.split(',')[3].split(':')[-1].split('}')[0]))
                iters_list.append(num_iters)
                kc_perturbs_list.append(kc_perturbation)
                km_perturbs_list.append(km_perturbation)

num_trials_data = pd.DataFrame({'num_iters': iters_list, 'km_perturb': km_perturbs_list, 'kc_perturb': kc_perturbs_list, 'loss': best_losses, 'k1':k1s, 'k2':k2s, 'theta1':theta1s, 'theta2':theta2s})
num_trials_data.to_csv(ode_name + '_perturbation_grid.csv')

100%|██████████| 500/500 [00:10<00:00, 48.66trial/s, best loss: 0.05819677307583214] 


In [17]:
#Set architecture
ode_function = dual_control
ode_name = 'dc'
max_iters = [500]
num_tries = 3


def objective(param_values):
    #Integration conditions
    t = np.linspace(0, 5E4, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.])
    extra_options = {'old_api': False, 'user_data': param_values}
    ode_solver = ode('cvode', ode_function, **extra_options)
    solution = ode_solver.solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2)
    return loss

perturbed_percentages = [0.5, 0.75, 0.9, 0.95, 1, 1.05, 1.1, 1.25, 1.5]
global kc
global km
iters_list = []
km_perturbs_list = []; kc_perturbs_list = []
k1s = []; k2s = []; theta1s = []; theta2s = []
best_losses = []
for num_iters in max_iters:
    for n in range(num_tries):
        for kc_perturbation in perturbed_percentages:
            for km_perturbation in perturbed_percentages:
                if km_perturbation == 1 or kc_perturbation == 1:
                    kc = kc_perturbation*12.
                    km = km_perturbation*10.
                    best = fmin(objective, space, algo=tpe.suggest, max_evals=num_iters)
                    best_losses.append(objective(list(best.values())))
                    d = str(best)
                    k1s.append(float(d.split(',')[0].split(':')[-1]))
                    k2s.append(float(d.split(',')[1].split(':')[-1]))
                    theta1s.append(float(d.split(',')[2].split(':')[-1]))
                    theta2s.append(float(d.split(',')[3].split(':')[-1].split('}')[0]))
                    iters_list.append(num_iters)
                    kc_perturbs_list.append(kc_perturbation)
                    km_perturbs_list.append(km_perturbation)

num_trials_data = pd.DataFrame({'num_iters': iters_list, 'km_perturb': km_perturbs_list, 'kc_perturb': kc_perturbs_list, 'loss': best_losses, 'k1':k1s, 'k2':k2s, 'theta1':theta1s, 'theta2':theta2s})
num_trials_data.to_csv(ode_name + '_perturbation_grid_1s.csv')

100%|██████████| 500/500 [00:09<00:00, 52.55trial/s, best loss: 0.05904467651714636] 


In [18]:
#Set architecture
ode_function = upstream_repression
ode_name = 'ur'
max_iters = [500]
num_tries = 3


def objective(param_values):
    #Integration conditions
    t = np.linspace(0, 5E4, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.])
    extra_options = {'old_api': False, 'user_data': param_values}
    ode_solver = ode('cvode', ode_function, **extra_options)
    solution = ode_solver.solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2)
    return loss

perturbed_percentages = [0.5, 0.75, 0.9, 0.95, 1, 1.05, 1.1, 1.25, 1.5]
global kc
global km
iters_list = []
km_perturbs_list = []; kc_perturbs_list = []
k1s = []; k2s = []; theta1s = []; theta2s = []
best_losses = []
for num_iters in max_iters:
    for n in range(num_tries):
        for kc_perturbation in perturbed_percentages:
            for km_perturbation in perturbed_percentages:
                if km_perturbation == 1 or kc_perturbation == 1:
                    kc = kc_perturbation*12.
                    km = km_perturbation*10.
                    best = fmin(objective, space, algo=tpe.suggest, max_evals=num_iters)
                    best_losses.append(objective(list(best.values())))
                    d = str(best)
                    k1s.append(float(d.split(',')[0].split(':')[-1]))
                    k2s.append(float(d.split(',')[1].split(':')[-1]))
                    theta1s.append(float(d.split(',')[2].split(':')[-1]))
                    theta2s.append(float(d.split(',')[3].split(':')[-1].split('}')[0]))
                    iters_list.append(num_iters)
                    kc_perturbs_list.append(kc_perturbation)
                    km_perturbs_list.append(km_perturbation)

num_trials_data = pd.DataFrame({'num_iters': iters_list, 'km_perturb': km_perturbs_list, 'kc_perturb': kc_perturbs_list, 'loss': best_losses, 'k1':k1s, 'k2':k2s, 'theta1':theta1s, 'theta2':theta2s})
num_trials_data.to_csv(ode_name + '_perturbation_grid_1s.csv')

100%|██████████| 500/500 [00:08<00:00, 58.01trial/s, best loss: 0.06701865234708052]
